In [1]:
from linkage_map.multi_family_mapping_functions import *

In [39]:
import os.path
import pandas as pd
import collections

### Write a file listing all identified paralogs

In [3]:
stats_file_1 = os.path.join('linkage_map','chum_08.stats')
stats_file_2 = os.path.join('linkage_map','chum_01.stats')
stats_file_3 = os.path.join('linkage_map','chum_09.stats')

paralogs_file = os.path.join('linkage_map','chum_paralogs.txt')

In [4]:
paralogs = set()
for stats_file in [stats_file_1, stats_file_2, stats_file_3]:
    with open(stats_file) as INFILE:
        for line in INFILE:
            if line.strip().split()[2] not in ['AA_xx', "AB"]:
                paralogs.add(line.strip().split()[0])
paralogs = sorted([int(xx) for xx in paralogs])
with open(paralogs_file, 'w') as OUTFILE:
    for xx in sorted(list(paralogs)):
        OUTFILE.write('{}\n'.format(xx))

### Import genotype data from MST map input files:

In [5]:
linkage_map_file_1 = os.path.join('linkage_map','chum_08_mstmap.txt')
linkage_map_file_2 = os.path.join('linkage_map','chum_01_mstmap.txt')
linkage_map_file_3 = os.path.join('linkage_map','chum_09_mstmap.txt')

In [6]:
individuals_08, genotypes_at_locus_08 = import_MSTmap(linkage_map_file_1)
individuals_01, genotypes_at_locus_01 = import_MSTmap(linkage_map_file_2)
individuals_09, genotypes_at_locus_09 = import_MSTmap(linkage_map_file_3)

my_pd_genos_08 = prep_data_pandas(individuals_08, genotypes_at_locus_08)
my_pd_genos_01 = prep_data_pandas(individuals_01, genotypes_at_locus_01)
my_pd_genos_09  = prep_data_pandas(individuals_09,  genotypes_at_locus_09)

fam_08 = prepare_matrix(my_pd_genos_08)
fam_01 = prepare_matrix(my_pd_genos_01)
fam_09 = prepare_matrix(my_pd_genos_09)

In [7]:
def rename_loci_by_family(paralogs_file, fam_names, families):
    # check if each family listed in families is formatted as if returned from prepare matrix
    for afam in families:
        if not isinstance(afam, pd.core.frame.DataFrame):
            raise ValueError("families should be a pandas DataFrame")
        else:
            pass
    if len(fam_names) != len(families) :
        raise ValueError("names and families should have the same length")
    if not isinstance(fam_names, list ):
        raise ValueError("names should be a list")
    
    with open(paralogs_file) as INFILE: 
        paralogs = [yy.strip() for yy in INFILE.readlines()]
    # for each family, for each locus, if the locus is a paralog, append family-specific text to locus name
    # genotypes are unchanged
    #new_familes = list()
    for idx, afam in enumerate(families):
        old_locus_names = afam.columns.values.tolist()
        new_locus_names = []
        for xx in old_locus_names:
            base_name = xx[:-3]
            if base_name in paralogs:
                #print("{} is a paralog".format(base_name))
                new_name = "{}_{}_{}".format(base_name, fam_names[idx], xx[-2:])
            else: 
                new_name = base_name
            new_locus_names.append(new_name)
        afam.columns = new_locus_names
    return(families)

In [8]:
renamed_08, renamed_01, renamed_09 = rename_loci_by_family(paralogs_file = paralogs_file, 
    fam_names = ['chum_08', 'chum_01', 'chum_09'], families = [my_pd_genos_08, my_pd_genos_01, my_pd_genos_09])

In [9]:
renamed_08.head()

,10001,10003_chum_08_x1,10020,10029,10039,10043,1004_chum_08_x1,10061,10068_chum_08_x1,10068_chum_08_x2,...,Oke_lactb2-71,Oke_mgll-49,Oke_nc2b-148,Oke_pnrc2-78,Oke_sylc-90,Oke_thic-84,Oke_txnrd1-74,Oke_u0602-244,Oke_u217-172,Oke_zn593-152
CMUW10X08H_0002,1,2,2,1,1,2,1,2,1,2,...,2,1,2,2,1,2,2,1,1,2
CMUW10X08H_0003,2,1,2,2,1,1,1,2,2,2,...,2,2,2,2,1,1,2,2,1,1
CMUW10X08H_0005,2,2,1,1,1,2,1,2,1,2,...,1,2,1,2,1,2,2,2,2,2
CMUW10X08H_0006,2,2,2,1,2,1,1,2,2,1,...,1,2,1,2,2,2,1,1,2,2
CMUW10X08H_0011,1,1,1,2,2,1,2,2,1,1,...,1,1,2,1,2,1,2,2,2,2


In [10]:
###### this should be moved inside function
renamed_08t = renamed_08.transpose()
renamed_01t = renamed_01.transpose()
renamed_09t = renamed_09.transpose()

In [11]:
aa = pd.merge(left = renamed_08t, right = renamed_01t, how = 'outer', left_index =True, right_index =True)

In [12]:
bb = pd.merge(left = aa, right = renamed_09t, how = 'outer', left_index =True, right_index =True) 
###### end move into function

In [13]:
all_my_data, loci_all = prepare_matrix(bb.transpose())

In [14]:
def write_LEPmap(families, family_names, loci, genotypes, output_filename):
    with open(output_filename, 'w') as OUTFILE:
        header = "\t".join(["#family", 'name', 'sire', 'dam', 'sex', 'blank'] + loci) + "\n"
        OUTFILE.write(header)
        for fam_idx, fam in enumerate(families):
            fam_name = family_names[fam_idx]
            DAM_line = "\t".join([fam_name, fam_name + "_Dam", '0', '0', '2', '0'] + ['1 1' for xx in loci]) + "\n"
            SIRE_line = "\t".join([fam_name, fam_name + "_Sire", '0', '0', '1', '0'] + ['1 2' for xx in loci]) + "\n"
            OUTFILE.write(DAM_line)
            OUTFILE.write(SIRE_line)
            for ind in fam:
                ind_info = "\t".join([fam_name, ind, fam_name + "_Sire", fam_name + "_Dam", '0', '0'])
                ind_genotypes = genotypes.loc[ind]
                OUTFILE.write(ind_info + "\t" + "\t".join([str(xx) for xx in ind_genotypes]) + "\n")

In [20]:
fams = [individuals_08, individuals_01, individuals_09]
LEPmap_filename = os.path.join('linkage_map','LEPmap', 'with_paralogs', 'all_loci.lepmap')

In [21]:
my_genotypes = bb.transpose()
my_genotypes = my_genotypes.replace(to_replace = [np.NaN, 0, 1, 2 ], value = ['0 0', '0 0', '1 1', '1 2'])

In [22]:
my_genotypes.shape

(240, 12603)

In [23]:
pd.read_csv("linkage_map/LEPmap/with_paralogs/all_loci.lepmap", sep = '\t').shape

(246, 12522)

In [24]:
write_LEPmap(families = fams, family_names = ["fam_08", "fam_01", "fam_09"], 
             loci = loci_all, genotypes = my_genotypes, output_filename = LEPmap_filename)

In [25]:
print "java SeparateChromosomes data=/home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/all_loci.lepmap \
lodLimit = 10 sizeLimit = 3 \
> /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/all_loci.inital_chromosomes \
2> /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/all_loci.inital_chromosomes.log"

print "java JoinSingles /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/all_loci.inital_chromosomes \
lodLimit = 8 \
data=/home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/all_loci.lepmap \
> /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/all_loci.lod8_singles.chromosomes \
2> /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/all_loci.lod8_singles.chromosomes.log"

print "java JoinSingles /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/all_loci.lod8_singles.chromosomes \
lodLimit = 7 \
data=/home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/all_loci.lepmap \
> /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/all_loci.lod7_singles.chromosomes \
2> /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/all_loci.lod7_singles.chromosomes.log"

print "java JoinSingles /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/all_loci.lod7_singles.chromosomes \
lodLimit = 6 \
data=/home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/all_loci.lepmap \
> /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/all_loci.lod6_singles.chromosomes \
2> /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/all_loci.lod6_singles.chromosomes.log"

print "java JoinSingles /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/all_loci.lod6_singles.chromosomes \
lodLimit = 5 \
data=/home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/all_loci.lepmap \
> /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/all_loci.lod5_singles.chromosomes \
2> /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/all_loci.lod5_singles.chromosomes.log"

print "java JoinSingles /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/all_loci.lod5_singles.chromosomes \
lodLimit = 4 \
data=/home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/all_loci.lepmap \
> /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/all_loci.lod4_singles.chromosomes \
2> /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/all_loci.lod4_singles.chromosomes.log"

print "java JoinSingles /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/all_loci.lod4_singles.chromosomes \
lodLimit = 3.5 \
data=/home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/all_loci.lepmap \
> /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/all_loci.lod3-5_singles.chromosomes \
2> /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/all_loci.lod3-5_singles.chromosomes.log"


java SeparateChromosomes data=/home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/all_loci.lepmap lodLimit = 10 sizeLimit = 3 > /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/all_loci.inital_chromosomes 2> /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/all_loci.inital_chromosomes.log
java JoinSingles /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/all_loci.inital_chromosomes lodLimit = 8 data=/home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/all_loci.lepmap > /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/all_loci.lod8_singles.chromosomes 2> /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/all_loci.lod8_singles.chromosomes.log
java JoinSingles /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/all_loci.lod8_singles.chromosomes lodLimit = 7 data=/home/ipseg/Desktop/

In [28]:
def find_duplicate_names(paralogs_file, loci, LG_file, output_file):
    with open(paralogs_file) as INFILE: 
        paralogs = [yy.strip() for yy in INFILE.readlines()]
    #get list of the locus names
        #  given by loci
    #get list of LG assignments
    with open(LG_file) as INFILE:
        #skip first line
        next(INFILE)
        LG_assignments = [int(line.strip()) for line in INFILE]
    LG_of_locus = dict(zip(loci, LG_assignments))
    #print(paralogs)
    with open(output_file, 'w') as OUTFILE:
        OUTFILE.write("{}\t{}\t{}\t{}\n".format('catalog_name', 'family', 'copy', 'LG'))
        for locus in loci:
            catalog_name = locus.split("_")[0]
            if catalog_name in paralogs:
                family = locus.split("_")[2]
                copy = locus[-2:]
                OUTFILE.write("{}\t{}\t{}\t{}\n".format(catalog_name, family, copy, LG_of_locus[locus]))
            #list of all loci sharing base name
                sharing = [loc for loc in loci if locus.split("_")[0] == loc.split("_")[0]]
                sharing.remove(locus)
                agree_on_LG = sum([1 for loc in sharing if LG_of_locus[loc] == LG_of_locus[locus] ])
                disagree_on_LG = sum([1 for loc in sharing if LG_of_locus[loc] != LG_of_locus[locus] ])
                #print(locus, agree_on_LG, disagree_on_LG)

In [30]:
find_duplicate_names(paralogs_file = paralogs_file, 
    loci = loci_all, LG_file = "/home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/all_loci.lod5_singles.chromosomes", 
    output_file = '/home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/LG_congruence.tsv')

In [53]:
# this is not used!
def write_name_table(paralogs_file, loci, LG_file, output_file):
    with open(paralogs_file) as INFILE: 
        paralogs = [yy.strip() for yy in INFILE.readlines()]
    with open(LG_file) as INFILE:
        #skip first line
        next(INFILE)
        LG_assignments = [int(line.strip()) for line in INFILE]
    LG_of_locus = collections.defaultdict(int, zip(loci, LG_assignments))
    with open(output_file, 'w') as OUTFILE:
        OUTFILE.write("{}\t{}\t{}\t{}\n".format('catalog_name', 'LG_08_x1', 'LG_08_x2', 'LG_01_x1', 'LG_01_x2', 'LG_09_x1', 'LG_09_x2'))
        for locus in loci:
            catalog_name = locus.split("_")[0]
            if catalog_name in paralogs:
                family = locus.split("_")[2]
                copy = locus[-2:]
                OUTFILE.write("{}\t{}\t{}\t{}\t{}\n".format(catalog_name, LG_of_locus[locus]))
            #list of all loci sharing base name
                sharing = [loc for loc in loci if locus.split("_")[0] == loc.split("_")[0]]
                sharing.remove(locus)
                agree_on_LG = sum([1 for loc in sharing if LG_of_locus[loc] == LG_of_locus[locus] ])
                disagree_on_LG = sum([1 for loc in sharing if LG_of_locus[loc] != LG_of_locus[locus] ])
                #print(locus, agree_on_LG, disagree_on_LG)

In [54]:
#write_name_table(paralogs_file = paralogs_file,
#    loci = loci_all, LG_file = "/home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/all_loci.lod5_singles.chromosomes", 
#    output_file = '/home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/name_table.tsv')

In [55]:
def collapse_names(LG_08_x1, LG_08_x2, LG_01_x1, LG_01_x2, LG_09_x1, LG_09_x2):
    # take them in order given here
    possible_results = ['A','B','C','D','E','F']
    #possible_results.reverse()
    result = []
    
    # check for segmental
    # if segmental do not try to resolve
    for xx in ((LG_08_x1, LG_08_x2), (LG_01_x1, LG_01_x2), (LG_09_x1, LG_09_x2)):
        x1, x2 = xx
        if x1 == x2 and x1 != 0: # segmental
            result = possible_results

    else:
        mapping = dict()
        for assign in (LG_08_x1, LG_08_x2, LG_01_x1, LG_01_x2, LG_09_x1, LG_09_x2):
            if assign in mapping:
                result.append(mapping[assign])
            else:
                if assign == 0:
                     result.append(possible_results.pop(0))
                else:
                    mapping[assign] = possible_results.pop(0)
                    result.append(mapping[assign])
    return(result)  

In [56]:
#examples
print collapse_names(1,2,2,1,2,1)
print collapse_names(0,1,2,0,2,0)
print collapse_names(1,0,2,0,2,0)
print collapse_names(0,0,1,0,2,0)
print collapse_names(0,12,0,0,2,2)

['A', 'B', 'B', 'A', 'B', 'A']
['A', 'B', 'C', 'D', 'C', 'E']
['A', 'B', 'C', 'D', 'C', 'E']
['A', 'B', 'C', 'D', 'E', 'F']
['F', 'A', 'B', 'C', 'D', 'E', 'E']


In [57]:
def parse_LG_congruence_line(line):
    catalog_name, family, copy, LG = line.strip().split("\t")
    return(catalog_name, family, copy, LG)

In [58]:
def write_rename_table(LG_congruence_file, out_file):
    famLG_of_locus = collections.defaultdict(dict)
    with open(out_file, 'w') as OUTFILE:
        OUTFILE.write("{}\t{}\t{}\n".format("old_name", "new_name", "LG"))
        with open(LG_congruence_file) as PARALOG_LGS:
            #skip first line
            next(PARALOG_LGS)
            for line in PARALOG_LGS:
                catalog_name, family, copy, LG = parse_LG_congruence_line(line)
                famLG_of_locus[catalog_name][family, copy] = int(LG)
            for cn, famLG in famLG_of_locus.items():
                LG_08_x1 = famLG.get(('08', 'x1'), 0)
                LG_08_x2 = famLG.get(('08', 'x2'), 0)
                LG_01_x1 = famLG.get(('01', 'x1'), 0)
                LG_01_x2 = famLG.get(('01', 'x2'), 0)
                LG_09_x1 = famLG.get(('09', 'x1'), 0)
                LG_09_x2 = famLG.get(('09', 'x2'), 0)
                #print cn, [LG_08_x1, LG_08_x2, LG_01_x1, LG_01_x2, LG_09_x1, LG_09_x2], collapse_names(LG_08_x1, LG_08_x2, LG_01_x1, LG_01_x2, LG_09_x1, LG_09_x2)
                fam_names = ["{}_chum_08_x1", "{}_chum_08_x2", "{}_chum_01_x1", "{}_chum_01_x2", "{}_chum_09_x1", "{}_chum_09_x2"]
                collaped_names = collapse_names(LG_08_x1, LG_08_x2, LG_01_x1, LG_01_x2, LG_09_x1, LG_09_x2)
                LGs = [LG_08_x1, LG_08_x2, LG_01_x1, LG_01_x2, LG_09_x1, LG_09_x2]
                family_specific_names = [xx.format(cn) for xx in fam_names]
                for cnt, fsn in enumerate(family_specific_names):
                    OUTFILE.write("{}\t{}\t{}\n".format(fsn, cn+"_{}".format(collaped_names[cnt]), LGs[cnt]))

In [59]:
write_rename_table(LG_congruence_file =  "/home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/LG_congruence.tsv", 
    out_file = "/home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/rename_table.tsv")   

In [60]:
#get rename dict
rename_table = pd.read_table("/home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/rename_table.tsv")
rename_dict = dict(zip(rename_table.old_name,rename_table.new_name))

In [61]:
renamed_08t = renamed_08.rename(columns=rename_dict).transpose()
renamed_01t = renamed_01.rename(columns=rename_dict).transpose()
renamed_09t = renamed_09.rename(columns=rename_dict).transpose()
aa = pd.merge(left = renamed_08t, right = renamed_01t, how = 'outer', left_index =True, right_index =True)
bb = pd.merge(left = aa, right = renamed_09t, how = 'outer', left_index =True, right_index =True)

In [62]:
fams = [individuals_08, individuals_01, individuals_09]
LEPmap_filename = "/home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/collapsed_loci.lepmap"
my_genotypes = bb.transpose()
my_genotypes = my_genotypes.replace(to_replace = [np.NaN, 0, 1, 2 ], value = ['0 0', '0 0', '1 1', '1 2'])
write_LEPmap(families = fams, family_names = ["fam_08", "fam_01", "fam_09"], loci = my_genotypes.columns.values.tolist(),
    genotypes = my_genotypes, output_filename = LEPmap_filename)

In [63]:
# form linkage groups
print "java SeparateChromosomes data=/home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/collapsed_loci.lepmap \
lodLimit = 10 sizeLimit = 20 \
> /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/collapsed_loci.inital.chromosomes \
2> /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/collapsed_loci.inital.chromosomes.log"

print "java JoinSingles /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/collapsed_loci.inital.chromosomes \
lodLimit = 8 lodDifference=3 \
data=/home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/collapsed_loci.lepmap \
> /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/collapsed_loci.lod8_singles.chromosomes \
2> /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/collapsed_loci.lod8_singles.chromosomes.log"

print "java JoinSingles /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/collapsed_loci.lod8_singles.chromosomes \
lodLimit = 7 lodDifference=3 \
data=/home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/collapsed_loci.lepmap \
> /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/collapsed_loci.lod7_singles.chromosomes \
2> /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/collapsed_loci.lod7_singles.chromosomes.log"

print "java JoinSingles /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/collapsed_loci.lod7_singles.chromosomes \
lodLimit = 6 lodDifference=3 \
data=/home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/collapsed_loci.lepmap \
> /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/collapsed_loci.lod6_singles.chromosomes \
2> /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/collapsed_loci.lod6_singles.chromosomes.log"

print "java JoinSingles /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/collapsed_loci.lod6_singles.chromosomes \
lodLimit = 5 lodDifference=3 \
data=/home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/collapsed_loci.lepmap \
> /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/collapsed_loci.lod5_singles.chromosomes \
2> /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/collapsed_loci.lod5_singles.chromosomes.log"

java SeparateChromosomes data=/home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/collapsed_loci.lepmap lodLimit = 10 sizeLimit = 20 > /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/collapsed_loci.inital.chromosomes 2> /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/collapsed_loci.inital.chromosomes.log
java JoinSingles /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/collapsed_loci.inital.chromosomes lodLimit = 8 lodDifference=3 data=/home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/collapsed_loci.lepmap > /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/collapsed_loci.lod8_singles.chromosomes 2> /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/collapsed_loci.lod8_singles.chromosomes.log
java JoinSingles /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/collapsed_loc

# WORKING HERE

In [67]:
for xx in reversed(range(20,38)):
    print("java OrderMarkers /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/collapsed_loci.lod5_singles.chromosomes \
        data=/home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/collapsed_loci.lepmap \
        alpha=1 maxDistance=30 \
        chromosome={} \
        > /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/chr_{}.map \
        2> /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/chr_{}.map.log\n".format(xx, xx, xx)
        )

java OrderMarkers /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/collapsed_loci.lod5_singles.chromosomes         data=/home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/collapsed_loci.lepmap         alpha=1 maxDistance=30         chromosome=37         > /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/chr_37.map         2> /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/chr_37.map.log

java OrderMarkers /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/collapsed_loci.lod5_singles.chromosomes         data=/home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/collapsed_loci.lepmap         alpha=1 maxDistance=30         chromosome=36         > /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/chr_36.map         2> /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/chr_36.map.l

In [68]:
for xx in reversed(range(13,20)):
    print("java OrderMarkers /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/collapsed_loci.lod5_singles.chromosomes \
        data=/home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/collapsed_loci.lepmap \
        alpha=1 maxDistance=30 \
        chromosome={} \
        > /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/chr_{}.map \
        2> /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/chr_{}.map.log\n".format(xx, xx, xx)
        )

java OrderMarkers /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/collapsed_loci.lod5_singles.chromosomes         data=/home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/collapsed_loci.lepmap         alpha=1 maxDistance=30         chromosome=19         > /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/chr_19.map         2> /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/chr_19.map.log

java OrderMarkers /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/collapsed_loci.lod5_singles.chromosomes         data=/home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/collapsed_loci.lepmap         alpha=1 maxDistance=30         chromosome=18         > /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/chr_18.map         2> /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/chr_18.map.l

In [69]:
for xx in reversed(range(7,13)):
    print("java OrderMarkers /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/collapsed_loci.lod5_singles.chromosomes \
        data=/home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/collapsed_loci.lepmap \
        alpha=1 maxDistance=30 \
        chromosome={} \
        > /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/chr_{}.map \
        2> /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/chr_{}.map.log\n".format(xx, xx, xx)
        )

java OrderMarkers /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/collapsed_loci.lod5_singles.chromosomes         data=/home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/collapsed_loci.lepmap         alpha=1 maxDistance=30         chromosome=12         > /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/chr_12.map         2> /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/chr_12.map.log

java OrderMarkers /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/collapsed_loci.lod5_singles.chromosomes         data=/home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/collapsed_loci.lepmap         alpha=1 maxDistance=30         chromosome=11         > /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/chr_11.map         2> /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/chr_11.map.l

In [70]:
for xx in reversed(range(4,7)):
    print("java OrderMarkers /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/collapsed_loci.lod5_singles.chromosomes \
        data=/home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/collapsed_loci.lepmap \
        alpha=1 maxDistance=30 \
        chromosome={} \
        > /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/chr_{}.map \
        2> /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/chr_{}.map.log\n".format(xx, xx, xx)
        )

java OrderMarkers /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/collapsed_loci.lod5_singles.chromosomes         data=/home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/collapsed_loci.lepmap         alpha=1 maxDistance=30         chromosome=6         > /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/chr_6.map         2> /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/chr_6.map.log

java OrderMarkers /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/collapsed_loci.lod5_singles.chromosomes         data=/home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/collapsed_loci.lepmap         alpha=1 maxDistance=30         chromosome=5         > /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/chr_5.map         2> /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/chr_5.map.log

ja

In [71]:
for xx in reversed(range(2,4)):
    print("java OrderMarkers /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/collapsed_loci.lod5_singles.chromosomes \
        data=/home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/collapsed_loci.lepmap \
        alpha=1 maxDistance=30 \
        chromosome={} \
        > /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/chr_{}.map \
        2> /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/chr_{}.map.log\n".format(xx, xx, xx)
        )

java OrderMarkers /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/collapsed_loci.lod5_singles.chromosomes         data=/home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/collapsed_loci.lepmap         alpha=1 maxDistance=30         chromosome=3         > /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/chr_3.map         2> /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/chr_3.map.log

java OrderMarkers /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/collapsed_loci.lod5_singles.chromosomes         data=/home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/collapsed_loci.lepmap         alpha=1 maxDistance=30         chromosome=2         > /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/chr_2.map         2> /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/chr_2.map.log



In [72]:
for xx in reversed(range(1,2)):
    print("java OrderMarkers /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/collapsed_loci.lod5_singles.chromosomes \
        data=/home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/collapsed_loci.lepmap \
        alpha=1 maxDistance=30 \
        chromosome={} \
        > /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/chr_{}.map \
        2> /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/chr_{}.map.log\n".format(xx, xx, xx)
        )

java OrderMarkers /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/collapsed_loci.lod5_singles.chromosomes         data=/home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/collapsed_loci.lepmap         alpha=1 maxDistance=30         chromosome=1         > /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/chr_1.map         2> /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/chr_1.map.log



In [ ]:
for xx in (1, 7):
    print("java OrderMarkers /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/collapsed_loci.lod5_singles.chromosomes \
        data=/home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/collapsed_loci.lepmap \
        alpha=1 maxDistance=30 \
        chromosome={} \
        > /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/chr_{}.map \
        2> /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/chr_{}.map.log\n".format(xx, xx, xx)
        )

for xx in (1, 7, 25, 26, 31, 33):
    print("java OrderMarkers /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/collapsed_loci.lod5_singles.chromosomes \
        data=/home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/collapsed_loci.lepmap \
        alpha=.5 maxDistance=20 \
        chromosome={} \
        > /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/chr_{}.map \
        2> /home/ipseg/Desktop/waples/chum_populations/linkage_map/LEPmap/with_paralogs/chr_{}.map.log\n".format(xx, xx, xx)
        )